In [ ]:
# !pip install nibabel numpy laspy

## TRK to LAS

In [ ]:
import nibabel as nib
import numpy as np
import laspy
import os
import glob

def load_trk_files_from_folder(folder_path):
    # Find all .trk files in the specified folder
    trk_files = glob.glob(os.path.join(folder_path, "*.trk"))
    return trk_files

def calculate_orientation_color(point1, point2):
    # Calculate the vector between two points and map it to RGB color
    vector = point2 - point1
    norm = np.linalg.norm(vector)
    if norm == 0:
        return (255, 255, 255)  # Default color for zero-length vectors (white)
    direction = vector / norm
    color = (
        int((0.5 + 0.5 * direction[0]) * 255),
        int((0.5 + 0.5 * direction[1]) * 255),
        int((0.5 + 0.5 * direction[2]) * 255),
    )
    return color

def add_intermediate_points(point1, point2, point_radius):
    # Calculate the distance between point1 and point2
    distance = np.linalg.norm(point2 - point1)
    if distance <= point_radius:
        return [point1]  # No need to add extra points, return point1 only

    # Number of intermediate points needed
    num_extra_points = int(np.floor(distance / point_radius))
    direction = (point2 - point1) / distance

    # Generate intermediate points
    intermediate_points = [point1 + i * point_radius * direction for i in range(num_extra_points + 1)]
    return intermediate_points

def trk_to_las(trk_files, output_las_file, point_radius=0.1):
    # Initialize lists to hold point and color data
    x = []
    y = []
    z = []
    red = []
    green = []
    blue = []

    # Load each .trk file and extract streamline points
    for trk_file in trk_files:
        tractogram = nib.streamlines.load(trk_file)
        streamlines = tractogram.streamlines

        # Iterate through each streamline to collect points and colors
        for streamline in streamlines:
            color = [0, 0, 0]
            for i in range(len(streamline) - 1):
                point1 = streamline[i]
                point2 = streamline[i + 1]

                # Add intermediate points between point1 and point2
                intermediate_points = add_intermediate_points(point1, point2, point_radius)

                for j in range(len(intermediate_points) - 1):
                    p1 = intermediate_points[j]
                    p2 = intermediate_points[j + 1]

                    # Append p1 coordinates
                    x.append(p1[0])
                    y.append(p1[1])
                    z.append(p1[2])

                    # Calculate color based on orientation between p1 and p2
                    color = calculate_orientation_color(p1, p2)
                    red.append(color[0])
                    green.append(color[1])
                    blue.append(color[2])

            # Append the last point of the streamline
            last_point = streamline[-1]
            x.append(last_point[0])
            y.append(last_point[1])
            z.append(last_point[2])
            red.append(color[0])
            green.append(color[1])
            blue.append(color[2])

    # Create a LAS file and write the collected data
    header = laspy.LasHeader(point_format=3, version="1.2")
    header.offsets = [min(x), min(y), min(z)]
    header.scales = [0.01, 0.01, 0.01]  # Scaling for precision

    las_data = laspy.LasData(header)
    las_data.x = np.array(x)
    las_data.y = np.array(y)
    las_data.z = np.array(z)
    las_data.red = np.array(red, dtype=np.uint16)
    las_data.green = np.array(green, dtype=np.uint16)
    las_data.blue = np.array(blue, dtype=np.uint16)

    # Write to LAS file
    las_data.write(output_las_file)

In [ ]:
# Example usage
folder_path = "HCP_subject124422_100Kseeds"
las_file = "HCP_subject124422_100Kseeds_as_streamlines.las"
trk_to_las(load_trk_files_from_folder(folder_path), las_file)

In [ ]:
import subprocess

potree_folder = 'dti'
basename = os.path.basename(las_file).replace('.las', '')
output_folder = os.path.join(potree_folder, basename)
metadata_file = os.path.join(output_folder, 'metadata.json')
try:
    command_line = 'PotreeConverter'
    subprocess.run(
        [
            command_line, 
            las_file,
            '-o', output_folder,
            '--encoding', 'BROTLI',
            '--output-format', 'LAZ',
            '-p', 'index'
        ],
        stdout=subprocess.DEVNULL
    )
except Exception as e:
    print(e)